# Codenation - Data Science
<pre>Autor: Leonardo Simões</pre>

## Desafio 6 - Feature engineering


Neste desafio vamos praticar feature engineering, a arte de processar variáveis do data set a fim de torná-las mais adequadas aos algoritmos de ML e produzir melhores resultados, um dos processos mais importantes e trabalhosos de ML.

Utilizaremos o _data set_ [Countries of the world](https://www.kaggle.com/fernandol/countries-of-the-world), que contém dados sobre os 227 países do mundo com informações sobre tamanho da população, área, imigração e setores de produção.


### Objetivo

O objetivo deste desafio é adquirir conhecimento e prática nas ferramentas mais usuais de engenharia de variáveis. Com o domínio apropriado das técnicas básicas, como one-hot encoding, normalização e padroniação, o analista está mais bem preparado para conduzir uma etapa de preprocessamento dos dados que traga bons resultados da aplicação dos algoritmos de ML.


### Tópicos

Neste desafios nós vamos explorar:
- Feature engineering
- Processamento de texto

### Questões
<pre>
1) Quais são as regiões (variável Region) presentes no data set_? 
Retorne uma lista com as regiões únicas do _data set com os espaços à frente e atrás da string removidos 
(mas mantenha pontuação: ponto, hífen etc) e ordenadas em ordem alfabética. 

2) Discretizando a variável Pop_density em 10 intervalos com KBinsDiscretizer, seguindo o encode ordinal 
e estratégia quantile, quantos países se encontram acima do 90º percentil? Responda como um único escalar 
inteiro.

3) Se codificarmos as variáveis Region e Climate usando _one-hot encoding_, quantos novos atributos seriam 
criados? Responda como um único escalar.

4) Após aplicado o pipeline descrito, aplique o mesmo pipeline (ou ColumnTransformer) a outros dados. 
Qual o valor da variável Arable após o _pipeline_? 
Responda como um único float arredondado para três casas decimais.

5) Descubra o número de outliers da variável Net_migration segundo o método do _boxplot_, ou seja, 
usando a lógica: 𝑥∉[𝑄1−1.5×IQR,𝑄3+1.5×IQR]⇒𝑥 é outlier que se encontram no grupo inferior e no grupo superior.
Você deveria remover da análise as observações consideradas outliers segundo esse método? 
Responda como uma tupla de três elementos (outliers_abaixo, outliers_acima, removeria?) ((int, int, bool)).

6) Considere carregar as seguintes categorias e o dataset newsgroups:
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
Aplique CountVectorizer ao data set newsgroups e descubra o número de vezes que a palavra phone aparece 
no corpus. Responda como um único escalar.

7) Aplique TfidfVectorizer ao data set newsgroups e descubra o TF-IDF da palavra phone. 
Responda como um único escalar arredondado para três casas decimais.
</pre>



## _Setup_ geral

In [200]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.feature_extraction.text import (CountVectorizer, TfidfTransformer, TfidfVectorizer)

from sklearn.preprocessing import (
    OneHotEncoder, Binarizer, KBinsDiscretizer,
    MinMaxScaler, StandardScaler, PolynomialFeatures
)
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.datasets import load_digits, fetch_20newsgroups
from sklearn.feature_extraction.text import (
    CountVectorizer, TfidfTransformer, TfidfVectorizer
)

In [201]:
#%matplotlib inline

from IPython.core.pylabtools import figsize

figsize(12, 8)

sns.set()

In [202]:
countries = pd.read_csv("countries.csv")

In [203]:
new_column_names = [
    "Country", "Region", "Population", "Area", "Pop_density", "Coastline_ratio",
    "Net_migration", "Infant_mortality", "GDP", "Literacy", "Phones_per_1000",
    "Arable", "Crops", "Other", "Climate", "Birthrate", "Deathrate", "Agriculture",
    "Industry", "Service"
]

countries.columns = new_column_names

countries.head(5)

,Country,Region,Population,Area,Pop_density,Coastline_ratio,Net_migration,Infant_mortality,GDP,Literacy,Phones_per_1000,Arable,Crops,Other,Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",NaN,NaN,NaN


## Observações

Esse _data set_ ainda precisa de alguns ajustes iniciais. Primeiro, note que as variáveis numéricas estão usando vírgula como separador decimal e estão codificadas como strings. Corrija isso antes de continuar: transforme essas variáveis em numéricas adequadamente.

Além disso, as variáveis `Country` e `Region` possuem espaços a mais no começo e no final da string. Você pode utilizar o método `str.strip()` para remover esses espaços.

## Análise dos dados

Avaliando as colunas do dataframe por tipo e valores. 

In [204]:
countries.columns

Index(['Country', 'Region', 'Population', 'Area', 'Pop_density',
       'Coastline_ratio', 'Net_migration', 'Infant_mortality', 'GDP',
       'Literacy', 'Phones_per_1000', 'Arable', 'Crops', 'Other', 'Climate',
       'Birthrate', 'Deathrate', 'Agriculture', 'Industry', 'Service'],
      dtype='object')

In [205]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country           227 non-null    object 
 1   Region            227 non-null    object 
 2   Population        227 non-null    int64  
 3   Area              227 non-null    int64  
 4   Pop_density       227 non-null    object 
 5   Coastline_ratio   227 non-null    object 
 6   Net_migration     224 non-null    object 
 7   Infant_mortality  224 non-null    object 
 8   GDP               226 non-null    float64
 9   Literacy          209 non-null    object 
 10  Phones_per_1000   223 non-null    object 
 11  Arable            225 non-null    object 
 12  Crops             225 non-null    object 
 13  Other             225 non-null    object 
 14  Climate           205 non-null    object 
 15  Birthrate         224 non-null    object 
 16  Deathrate         223 non-null    object 
 1

As colunas 'Country' e 'Region' serão formatadas para remover espaços no início e fim de cada valor.

In [206]:
colunas_qualitativas = ['Country', 'Region']

In [207]:
for coluna in colunas_qualitativas:
    countries[coluna] = countries[coluna].apply(lambda s : s.strip())

countries[colunas_qualitativas].head()

,Country,Region
0,Afghanistan,ASIA (EX. NEAR EAST)
1,Albania,EASTERN EUROPE
2,Algeria,NORTHERN AFRICA
3,American Samoa,OCEANIA
4,Andorra,WESTERN EUROPE


As colunas que são do tipo 'object' ('string') que representa valores float serão formatadas alterando o separador ',' para '.', e depois alterando seu tipo para 'float64'.

In [208]:
colunas_quantitativas_string = countries.select_dtypes(include=['object']).columns[2:]
colunas_quantitativas_string

Index(['Pop_density', 'Coastline_ratio', 'Net_migration', 'Infant_mortality',
       'Literacy', 'Phones_per_1000', 'Arable', 'Crops', 'Other', 'Climate',
       'Birthrate', 'Deathrate', 'Agriculture', 'Industry', 'Service'],
      dtype='object')

In [209]:
for coluna in colunas_quantitativas_string:
    countries[coluna] = countries[coluna].str.replace(',', '.')
    countries[coluna] = countries[coluna].astype('float64')

As colunas colunas com valores 'int64' serão convertidas para 'float64' a fim de facilitar alguns cálculos posteriores.

In [210]:
colunas_quantitativas_int = countries.select_dtypes(include=['int64']).columns
colunas_quantitativas_int

Index(['Population', 'Area'], dtype='object')

In [211]:
for coluna in colunas_quantitativas_int:
    countries[coluna] = countries[coluna].astype('float64')

O valores ausentes NaN de colunas quantitativas serão substituídos por 0. 

In [212]:
colunas_quantitativas = countries.select_dtypes(include=['float64', 'int64']).columns
colunas_quantitativas

Index(['Population', 'Area', 'Pop_density', 'Coastline_ratio', 'Net_migration',
       'Infant_mortality', 'GDP', 'Literacy', 'Phones_per_1000', 'Arable',
       'Crops', 'Other', 'Climate', 'Birthrate', 'Deathrate', 'Agriculture',
       'Industry', 'Service'],
      dtype='object')

In [213]:
len(colunas_quantitativas)

18

## Questão 1

Quais são as regiões (variável `Region`) presentes no _data set_? Retorne uma lista com as regiões únicas do _data set_ com os espaços à frente e atrás da string removidos (mas mantenha pontuação: ponto, hífen etc) e ordenadas em ordem alfabética.

In [214]:
def q1():
    regions = countries.Region.unique()
    regions = np.sort(regions)
    
    return list(regions)

q1()

['ASIA (EX. NEAR EAST)',
 'BALTICS',
 'C.W. OF IND. STATES',
 'EASTERN EUROPE',
 'LATIN AMER. & CARIB',
 'NEAR EAST',
 'NORTHERN AFRICA',
 'NORTHERN AMERICA',
 'OCEANIA',
 'SUB-SAHARAN AFRICA',
 'WESTERN EUROPE']

## Questão 2

Discretizando a variável `Pop_density` em 10 intervalos com `KBinsDiscretizer`, seguindo o encode `ordinal` e estratégia `quantile`, quantos países se encontram acima do 90º percentil? Responda como um único escalar inteiro.

In [215]:
def q2():
    pop_density = countries[['Pop_density']]
    discretizer = KBinsDiscretizer(n_bins = 10, encode = 'ordinal', strategy = 'quantile')
    discretizer.fit(pop_density)
    scores = discretizer.transform(pop_density)
    
    return int((scores >= 9).sum())

q2()

23

# Questão 3

Se codificarmos as variáveis `Region` e `Climate` usando _one-hot encoding_, quantos novos atributos seriam criados? Responda como um único escalar.

In [216]:
def q3():
    countries_r_c = countries.copy()
    countries_r_c = countries_r_c[['Region', 'Climate']]
    countries_r_c['Region'] = countries_r_c['Region'].fillna("")
    countries_r_c['Climate'] = countries_r_c['Climate'].fillna(0)
    
    encoder = OneHotEncoder(sparse = False).fit_transform(countries_r_c[['Region', 'Climate']]) 
    
    return int(encoder.shape[1])

q3()

18

## Questão 4

Aplique o seguinte _pipeline_:

1. Preencha as variáveis do tipo `int64` e `float64` com suas respectivas medianas.
2. Padronize essas variáveis.

Após aplicado o _pipeline_ descrito acima aos dados (somente nas variáveis dos tipos especificados), aplique o mesmo _pipeline_ (ou `ColumnTransformer`) ao dado abaixo. Qual o valor da variável `Arable` após o _pipeline_? Responda como um único float arredondado para três casas decimais.

In [217]:
test_country = [
    'Test Country', 'NEAR EAST', -0.19032480757326514,
    -0.3232636124824411, -0.04421734470810142, -0.27528113360605316,
    0.13255850810281325, -0.8054845935643491, 1.0119784924248225,
    0.6189182532646624, 1.0074863283776458, 0.20239896852403538,
    -0.043678728558593366, -0.13929748680369286, 1.3163604645710438,
    -0.3699637766938669, -0.6149300604558857, -0.854369594993175,
    0.263445277972641, 0.5712416961268142
]

In [218]:
len(test_country[2:])

18

In [219]:
len(colunas_quantitativas)

18

In [220]:
countries_copy = countries[colunas_quantitativas].fillna(0)

In [221]:
def q4():
    num_pipeline = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")),
                                   ('standard_scaler', StandardScaler())])
    pipeline_transformation = num_pipeline.fit_transform(countries[colunas_quantitativas])
    test_country_transform = num_pipeline.transform([test_country[2:]])
    
    return round(test_country_transform[:,countries.columns.get_loc("Arable")-2][0],3)

q4()

-1.047

## Questão 5

Descubra o número de _outliers_ da variável `Net_migration` segundo o método do _boxplot_, ou seja, usando a lógica:

$$x \notin [Q1 - 1.5 \times \text{IQR}, Q3 + 1.5 \times \text{IQR}] \Rightarrow x \text{ é outlier}$$

que se encontram no grupo inferior e no grupo superior.

Você deveria remover da análise as observações consideradas _outliers_ segundo esse método? Responda como uma tupla de três elementos `(outliers_abaixo, outliers_acima, removeria?)` ((int, int, bool)).

In [222]:
countries['Net_migration']

0      23.06
1      -4.93
2      -0.39
3     -20.71
4       6.60
       ...  
222     2.98
223      NaN
224     0.00
225     0.00
226     0.00
Name: Net_migration, Length: 227, dtype: float64

In [223]:
def q5():
    net_migrations = countries['Net_migration'].dropna()
    q1, q3 = np.quantile(net_migrations, .25), np.quantile(net_migrations, .75)
    iqr = q3 - q1

    non_outlier_interval_iqr = [q1 - 1.5 * iqr, q3 + 1.5 * iqr]

    outliers_above = net_migrations[net_migrations > non_outlier_interval_iqr[1]]
    outliers_below = net_migrations[net_migrations < non_outlier_interval_iqr[0]]
    
    return (len(outliers_below), len(outliers_above), False)
q5()

(24, 26, False)

## Questão 6
Para as questões 6 e 7 utilize a biblioteca `fetch_20newsgroups` de datasets de test do `sklearn`

Considere carregar as seguintes categorias e o dataset `newsgroups`:

```
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
```


Aplique `CountVectorizer` ao _data set_ `newsgroups` e descubra o número de vezes que a palavra _phone_ aparece no corpus. Responda como um único escalar.

In [224]:
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)

In [225]:
def q6():
    count_vectorizer = CountVectorizer() 
    newsgroup_counts = count_vectorizer.fit_transform(newsgroup.data)
    words = pd.DataFrame(newsgroup_counts.toarray(), columns=count_vectorizer.get_feature_names())
    
    return words['phone'].sum()

q6()

213

## Questão 7

Aplique `TfidfVectorizer` ao _data set_ `newsgroups` e descubra o TF-IDF da palavra _phone_. Responda como um único escalar arredondado para três casas decimais.

In [226]:
def q7():
    tfidf_vectorizer = TfidfVectorizer()
    newsgroups_tfidf_vectorized = tfidf_vectorizer.fit_transform(newsgroup.data)
    newsgroups_tfidf = pd.DataFrame(newsgroups_tfidf_vectorized.toarray(), columns=tfidf_vectorizer.get_feature_names())
    newsgroups_tfidf_phone = round(newsgroups_tfidf['phone'].sum(), 3)
    return float(newsgroups_tfidf_phone)

q7()

8.888